In [5]:
import sqlite3
import pandas as pd

db = sqlite3.connect('AMS.db')
query = 'select * from Order_Table'
df = pd.read_sql(query, con = db)
df['Date']=pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
dfMonth = df[(df['Year']==2019) & (df['Month']==12)]
dfOrderPL = dfMonth.groupby(['Date','OrderID'])[['SettleAmt']].sum()
dfOrderPL.reset_index()

,Date,OrderID,SettleAmt
0,2019-12-02,1,491.6
1,2019-12-02,2,101.6
2,2019-12-03,1,1691.6
3,2019-12-05,1,-564.4
4,2019-12-05,2,345.6
5,2019-12-05,3,95.6
6,2019-12-05,4,918.4
7,2019-12-09,1,975.6
8,2019-12-09,2,-774.4
9,2019-12-09,3,-554.4


In [2]:
dfMonth.groupby(['Date'])[['SettleAmt']].sum()

,SettleAmt
Date,
2019-12-02,593.2
2019-12-03,1691.6
2019-12-05,795.2
2019-12-09,-353.2
2019-12-10,171.2


In [3]:
df.groupby(['Year','Month'])[['SettleAmt']].sum()

SettleAmt
Year Month           
2019 11        2037.6
     12        2898.0

In [6]:
import pandas as pd
import portfolio
import sqlite3

db = sqlite3.connect('AMS.db')

dt = '2019-06-13'
account = ('CMS','CITIC')
df_trans = portfolio.get_trans(dt, account)

query = "SELECT SymbolCode, AssetClass FROM Symbol_Table"
df_symbol = pd.read_sql(query, con=db)

df = pd.merge(df_trans, df_symbol, on='SymbolCode', how='left')

df

,ID,AccountID,Date,Time,SymbolCode,SymbolName,OrderID,TradeID,OrderType,OpenClose,CurTrade,CurSettle,Price,Qty,Commission,TradeAmt,SettleAmt,AssetClass
0,97,CITIC,2018-12-31,00:00:00,CNY,人民币,0,0,Deposit,,CNY,CNY,1.000,115547.00,0.00,115547.00,115547.00,CASH
1,98,CITIC,2018-12-31,00:00:00,510500.SS,500ETF,0,0,Buy,,CNY,CNY,4.870,5000.00,0.00,24350.00,-24350.00,EQUITY
2,99,CITIC,2018-12-31,00:00:00,600177.SS,雅戈尔,0,0,Buy,,CNY,CNY,8.593,2000.00,0.00,17186.00,-17186.00,EQUITY
3,100,CITIC,2018-12-31,00:00:00,601088.SS,中国神华,0,0,Buy,,CNY,CNY,11.275,2600.00,0.00,29315.00,-29315.00,EQUITY
4,101,CITIC,2018-12-31,00:00:00,000726.SZ,鲁泰A,0,0,Buy,,CNY,CNY,11.174,4000.00,0.00,44696.00,-44696.00,EQUITY
5,102,CITIC,2018-12-31,00:00:00,CNY,人民币,0,0,Deposit,,CNY,CNY,1.000,61575.92,0.00,61575.92,61575.92,CASH
6,103,CMS,2018-12-31,00:00:00,CNY,人民币,0,0,Deposit,,CNY,CNY,1.000,93273.60,0.00,93273.60,93273.60,CASH
7,104,CMS,2018-12-31,00:00:00,512880.SS,证券ETF,0,0,Buy,,CNY,CNY,0.741,40000.00,0.00,29640.00,-29640.00,EQUITY
8,105,CMS,2018-12-31,00:00:00,600030.SS,中信证券,0,0,Buy,,CNY,CNY,19.136,1000.00,0.00,19136.00,-19136.00,EQUITY
9,106,CMS,2018-12-31,00:00:00,601166.SS,兴业银行,0,0,Buy,,CNY,CNY,15.892,2800.00,0.00,44497.60,-44497.60,EQUITY
